## Train a  Bernoulli-Bernoulli RBM (BBRBM) to fit XOR calculations

In [1]:
import numpy as np
import timeit
from tfrbm.bbrbm import BBRBM

In [2]:
bm = BBRBM(n_visible=3,n_hidden=13)

### dataset[i][0] xor dataset[i][1] = dataset[i][2]

In [3]:
dataset = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]])

### Train and predict same data in same epochs
x is the correct data to train the model  
x2 has wrong xor answer ( the third digit )  
x2 would be reconstructed to x after training

In [4]:
for i in range(4):
    x = dataset[i:i+1].copy()
    x2 = dataset[i:i+1].copy()
    x2[0,2] = 1-x2[0,2]
    print "x:",x
    print "x2:",x2
    print "Training model with x"
    err = bm.fit(x,n_epoches=500)
    print "model reconstructed x:",np.round(bm.reconstruct(x))
    print "model reconstructed x2:",np.round(bm.reconstruct(x2))
    print "Training done"

    positions_to_predict = [2]
    prediction = bm.predict(x,positions_to_predict=positions_to_predict) # best result, energy for all possibilities
    print "model predicting x third digit to be:",prediction[0].tolist()
    prediction = bm.predict(x2,positions_to_predict=positions_to_predict)
    print "model predicting x2 third digit to be:",prediction[0].tolist()
    print "------------------------------------"

x: [[0 0 0]]
x2: [[0 0 1]]
Training model with x
model reconstructed x: [[0. 0. 0.]]
model reconstructed x2: [[0. 0. 0.]]
Training done
model predicting x third digit to be: [0]
model predicting x2 third digit to be: [0]
------------------------------------
x: [[0 1 1]]
x2: [[0 1 0]]
Training model with x
model reconstructed x: [[0. 1. 1.]]
model reconstructed x2: [[0. 1. 1.]]
Training done
model predicting x third digit to be: [1]
model predicting x2 third digit to be: [1]
------------------------------------
x: [[1 0 1]]
x2: [[1 0 0]]
Training model with x
model reconstructed x: [[1. 0. 1.]]
model reconstructed x2: [[1. 0. 1.]]
Training done
model predicting x third digit to be: [1]
model predicting x2 third digit to be: [1]
------------------------------------
x: [[1 1 0]]
x2: [[1 1 1]]
Training model with x
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
Training done
model predicting x third digit to be: [0]
model predicting x2 third digit to be: [0]
-----

### All x variations would be reconstructured to the last training data [1, 1, 0]

In [5]:
for i in range(4):
    x = dataset[i:i+1].copy()
    x2 = dataset[i:i+1].copy()
    x2[0,2] = 1-x2[0,2]
    print "x:",x
    print "x2:",x2
    
    positions_to_predict = [2]
    print "model reconstructed x:",np.round(bm.reconstruct(x))
    print "model reconstructed x2:",np.round(bm.reconstruct(x2))
    print "------------------------------------"

x: [[0 0 0]]
x2: [[0 0 1]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[0 1 1]]
x2: [[0 1 0]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[1 0 1]]
x2: [[1 0 0]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------
x: [[1 1 0]]
x2: [[1 1 1]]
model reconstructed x: [[1. 1. 0.]]
model reconstructed x2: [[1. 1. 0.]]
------------------------------------


### Majority vote data would be the data reconstruction target
new fit() on top of the previous model will fit the model to the new training set

In [6]:
training_data = np.array([[0,0,0],[0,0,1]])
training_data = np.repeat(training_data, [8,2],axis=0)
print "training_data:",training_data
start = timeit.default_timer()
err = bm.fit(training_data,n_epoches=500)
stop = timeit.default_timer()
print 'Time: ', stop - start, "sec"
print "model reconstructed [0,0,0]:",np.round(bm.reconstruct(training_data[0:1]))
print "model reconstructed [0,0,1]:",np.round(bm.reconstruct(training_data[-2:-1]))
positions_to_predict = [2]
prediction = bm.predict(training_data[0:1],positions_to_predict=positions_to_predict)
print "model predicting [0,0,0] third digit to be:",prediction[0].tolist()

training_data: [[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]]
Time:  0.415551900864 sec
model reconstructed [0,0,0]: [[0. 0. 0.]]
model reconstructed [0,0,1]: [[0. 0. 0.]]
model predicting [0,0,0] third digit to be: [0]


### model would be overwritten with new training data
Model may not reconstruct to the next new majority [0,0,1] because of the previous trainings.  
But still can predict 0 xor 0 to be 1 from free energy comparison

Note here that the training process is stochastic, so different runs may lead to different results  
See tfrbm/bbrbm.py `line 12` `... sample_bernoulli ...`

In [7]:
training_data = np.array([[0,0,0],[0,0,1]])
training_data = np.repeat(training_data, [2,8],axis=0)
print "training_data:",training_data
start = timeit.default_timer()
err = bm.fit(training_data,n_epoches=500)
stop = timeit.default_timer()
print 'Time: ', stop - start, "sec"
print "model reconstructed [0,0,0]:",np.round(bm.reconstruct(training_data[0:1]))
print "model reconstructed [0,0,1]:",np.round(bm.reconstruct(training_data[-2:-1]))
positions_to_predict = [2]
prediction = bm.predict(training_data[0:1],positions_to_predict=positions_to_predict)
print "model predicting [0,0,0] third digit to be:",prediction[0].tolist()

training_data: [[0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]]
Time:  0.403935909271 sec
model reconstructed [0,0,0]: [[0. 0. 0.]]
model reconstructed [0,0,1]: [[0. 0. 1.]]
model predicting [0,0,0] third digit to be: [1]


### More votes from [0,0,1] can lead reconstruction of [0,0,0] to [0,0,1] with higher probabilities
Predicts still get correct answers

In [8]:
training_data = np.array([[0,0,0],[0,0,1]])
training_data = np.repeat(training_data, [2,108],axis=0)

start = timeit.default_timer()
err = bm.fit(training_data,n_epoches=500)
stop = timeit.default_timer()
print 'Time: ', stop - start, "sec"
print "model reconstructed [0,0,0]:",np.round(bm.reconstruct(training_data[0:1]))
print "model reconstructed [0,0,1]:",np.round(bm.reconstruct(training_data[-2:-1]))
positions_to_predict = [2]
prediction = bm.predict(training_data[0:1],positions_to_predict=positions_to_predict)
print "model predicting [0,0,0] third digit to be:",prediction[0].tolist()
prediction = bm.predict(training_data[-2:-1],positions_to_predict=positions_to_predict)
print "model predicting [0,0,1] third digit to be:",prediction[0].tolist()

Time:  4.59761214256 sec
model reconstructed [0,0,0]: [[0. 0. 1.]]
model reconstructed [0,0,1]: [[0. 0. 1.]]
model predicting [0,0,0] third digit to be: [1]
model predicting [0,0,1] third digit to be: [1]


### If training data has the same occurrence, both reconstructions occur

In [9]:
training_data = np.array([[0,0,0],[0,0,1]])
training_data = np.repeat(training_data, [88,88],axis=0)

start = timeit.default_timer()
err = bm.fit(training_data,n_epoches=500)
stop = timeit.default_timer()
print 'Time: ', stop - start, "sec"
print "model reconstructed [0,0,0]:",np.round(bm.reconstruct(training_data[0:1]))
print "model reconstructed [0,0,1]:",np.round(bm.reconstruct(training_data[-2:-1]))
print "model reconstructed [0,0,-1]:",np.round(bm.reconstruct([[0,0,-1]]))
print "model reconstructed [0,0,100]:",np.round(bm.reconstruct([[0,0,100]]))

Time:  7.8404211998 sec
model reconstructed [0,0,0]: [[0. 0. 0.]]
model reconstructed [0,0,1]: [[0. 0. 1.]]
model reconstructed [0,0,-1]: [[0. 0. 0.]]
model reconstructed [0,0,100]: [[0. 0. 1.]]
